# 03_ai_agent_analysis_part1_fixed — Config & API 拡張（統合版）
このノートブックは、**元コードを温存**したまま、以下のセルを**追記**して機能拡張します：
- セル0: Config読み込みとマージ (`load_configuration`)
- セル3(パッチ): DB/LLM設定のConfig反映 + GPU自動検出 + vLLM→Ollamaフォールバック
- セルX: 偽陰性分析の閾値をConfigから取得（低確率領域分析は必ず実行）
- 最終セル: Controller API (`agent_minimal`)
※ 元セルは変更・削除せず、**実行順序で上書き適用**します（副作用回避）。


In [1]:
# === Cell 0 (02以降 共通): レジストリから解決して paths を読む ===
import run_id_registry as runreg
rid = runreg.bootstrap()  # env→ファイル(artifacts/_current/run_id.txt)→Part3→latest→新規 の順で解決

import importlib
import _compat.paths as paths
importlib.reload(paths)
importlib.reload(paths)
print("[NX] RUN_ID =", rid, "| paths.RUN_ID =", paths.RUN_ID)


[NX] RUN_ID = 20251030_004055 | paths.RUN_ID = 20251030_004055


In [2]:

# === セル0: Config読み込みとマージ (追記) ===
from __future__ import annotations
from typing import Optional, Dict, Any, Tuple
import os, json
from pathlib import Path
try:
    import yaml  # optional
except Exception:
    yaml = None

def _deep_update(base: dict, override: dict) -> dict:
    base = dict(base or {})
    for k, v in (override or {}).items():
        if isinstance(v, dict) and isinstance(base.get(k), dict):
            base[k] = _deep_update(base[k], v)
        else:
            base[k] = v
    return base

def load_configuration(
    config_path: Optional[str] = None,
    cfg_override: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]:
    defaults = {
        "system": {
            "cert_only_mode": False,
            "enable_brand_keywords": True,
            "gpu_auto_detect": True,
            "development_mode": False,
            "seed": 42,
        },
        "llm": {
            "provider": "vllm",  # "vllm" | "ollama"
            "vllm_base_url": "http://192.168.100.71:30000/v1",
            "vllm_model": "Qwen/Qwen3-14B-FP8",
            "ollama_base_url": "http://localhost:11434/v1",
            "ollama_model": "qwen3:14b",
            "fallback_enabled": True,
            "temperature": 0.1,
            "max_tokens": 2000,
        },
        "db": {
            "dbname": "rapids_data",
            "user": "postgres",
            "password": "asomura",
            "host": "localhost",
            "port": "5432",
            "timeout_s": 30,
            "read_only": True,
        },
        "analysis": {
            "fn_threshold_low": 0.2,
            "fn_threshold_high": 0.7,
            "enable_detailed_stats": True,
        },
    }
    cfg = dict(defaults)

    # 2) config file (JSON/YAML) if exists
    def _read_config_file(p: str) -> dict:
        p = str(p)
        if not p or not os.path.exists(p):
            return {}
        try:
            text = Path(p).read_text(encoding="utf-8")
            if p.endswith((".yml",".yaml")) and yaml is not None:
                return yaml.safe_load(text) or {}
            return json.loads(text)
        except Exception as e:
            print(f"⚠️ 設定ファイル読み込みに失敗: {p} ({e})")
            return {}
    if config_path:
        cfg = _deep_update(cfg, _read_config_file(config_path))

    # 3) env vars
    env_map = {
        "DEV_MODE": ("system", "development_mode"),
        "LLM_TYPE": ("llm", "provider"),
        "VLLM_BASE_URL": ("llm", "vllm_base_url"),
        "PGDATABASE": ("db", "dbname"),
        "PGUSER": ("db", "user"),
        "PGPASSWORD": ("db", "password"),
    }
    for env, path in env_map.items():
        if env in os.environ and os.environ[env] != "":
            val = os.environ[env]
            low = val.lower()
            if low in ("true","false"):
                val = (low == "true")
            node = cfg
            for key in path[:-1]:
                node = node.setdefault(key, {})
            node[path[-1]] = val

    # 4) override
    if cfg_override:
        cfg = _deep_update(cfg, cfg_override)

    # 5) validation & warnings
    if cfg["system"].get("cert_only_mode", False):
        print("⚠️ 注意: cert_only_mode=True ですが、ブランド偽装検出は研究の核心機能のため実行します。")
        cfg["system"]["enable_brand_keywords"] = True

    provider = str(cfg["llm"].get("provider","vllm")).lower()
    if provider not in ("vllm","ollama"):
        print(f"⚠️ 未知のLLMプロバイダー '{provider}' -> 'vllm' に矯正します")
        cfg["llm"]["provider"] = "vllm"

    for req in ("dbname","user","host","port"):
        if not str(cfg["db"].get(req,"")).strip():
            raise ValueError(f"DB接続情報が不足しています: db.{req}")

    globals()["cfg"] = cfg
    print("✅ Config loaded (provider={provider}, gpu_auto_detect={gpu})".format(
        provider=cfg["llm"]["provider"],
        gpu=cfg["system"]["gpu_auto_detect"],
    ))
    return cfg


In [3]:
#!/usr/bin/env python
# coding: utf-8

# ## Part 1 — MLモデルが見逃したフィッシングサイトをAIエージェント（Qwen3-14B）で検出
# 
# ※本パートは原文のセルを**一字一句変更せず**収録しています。追加はこの見出しと入出力スロットのみです。

# In[1]:


# === IO PATHS guard (define RUN_ID, HANDOFF_DIR, etc.) ===
import os
from datetime import datetime
from pathlib import Path

RUN_ID = os.environ.get("RUN_ID") or datetime.now().strftime("%Y-%m-%d_%H%M%S")
ARTIFACTS = Path("artifacts") / RUN_ID
RAW = ARTIFACTS / "raw"; PROCESSED = ARTIFACTS / "processed"; MODELS = ARTIFACTS / "models"
RESULTS = ARTIFACTS / "results"; HANDOFF = ARTIFACTS / "handoff"; LOGS = ARTIFACTS / "logs"; TRACES = ARTIFACTS / "traces"
for _p in [RAW, PROCESSED, MODELS, RESULTS, HANDOFF, LOGS, TRACES]:
    _p.mkdir(parents=True, exist_ok=True)

# 文字列パス（互換）
RAW_DIR=str(RAW); PROCESSED_DIR=str(PROCESSED); MODELS_DIR=str(MODELS)
RESULTS_DIR=str(RESULTS); HANDOFF_DIR=str(HANDOFF); LOGS_DIR=str(LOGS); TRACES_DIR=str(TRACES)

print("✅ IO guard ready ->", HANDOFF_DIR)


# In[2]:


# === 02の仕様に基づく「最新セッション自動選択」ラッパー（追加セル） ===
from pathlib import Path
import re, json
from datetime import datetime

def _parse_session_id(name: str):
    # 02は "%Y-%m-%d_%H%M%S" 形式を採用
    m = re.match(r"^(\d{4})-(\d{2})-(\d{2})_(\d{6})$", name)
    if not m:
        return None
    try:
        return datetime.strptime(name, "%Y-%m-%d_%H%M%S")
    except ValueError:
        return None

def _exists_all(sess_dir: Path):
    # 02の成果物（最低限）
    need = ["xgboost_model.pkl", "scaler.pkl"]
    missing = [n for n in need if not (sess_dir / n).exists()]
    return (len(missing) == 0), missing

models_root = Path("models")
if not models_root.exists():
    raise FileNotFoundError("models/ が見つかりません。まず 02_xgboost_training_evaluation.py を実行してください。")

candidates = []
for d in models_root.glob("*"):
    if d.is_dir():
        ts = _parse_session_id(d.name)
        if ts:
            ok, missing = _exists_all(d)
            if ok:
                # xgboost_model.pkl の mtime も参考に
                model_pkl = d / "xgboost_model.pkl"
                candidates.append((ts, model_pkl.stat().st_mtime, d))

if not candidates:
    raise FileNotFoundError("02の仕様に合致するモデルが見つかりません。\n"
                            "models/YYYY-MM-DD_HHMMSS/{xgboost_model.pkl, scaler.pkl} を確認してください。")

# 日時→mtime の優先順で最新を選択
candidates.sort(key=lambda t: (t[0], t[1]))
latest_dir = candidates[-1][2]
session_id = latest_dir.name  # ここで 03 の後続セルが使う session_id を上書き

# 03 が参照するであろうパスを上書き定義（存在チェックは緩く。無いものは後段の原文ロジックに委ねる）
model_path = str(latest_dir / "xgboost_model.pkl")
scaler_path = str(latest_dir / "scaler.pkl")
feature_order_path = str(latest_dir / "feature_order.json")  # ある場合想定
brand_keywords_path = str(latest_dir / "brand_keywords.json")  # ある場合想定

print("✅ 02仕様に基づき最新セッションを自動選択:")
print("  session_id  :", session_id)
print("  model_path  :", model_path)
print("  scaler_path :", scaler_path)
print("  feature_order_path :", feature_order_path, ("(存在しません)" if not Path(feature_order_path).exists() else ""))
print("  brand_keywords_path:", brand_keywords_path, ("(存在しません)" if not Path(brand_keywords_path).exists() else ""))

# 参考: 偽陰性ピクル（必要なら後段で参照）
results_pkl = Path("results") / session_id / "false_negatives.pkl"
if results_pkl.exists():
    print("  results FN pkl:", results_pkl)
else:
    print("  results FN pkl: (見つかりません)")

# 以降は 03 の原文セル（joblib.load 等）をそのまま実行してください


# In[3]:


"""
プログラム名: 03_ai_agent_analysis.ipynb
概要: MLモデルが見逃したフィッシングサイトをAIエージェント（Qwen3-14B）で検出
目的: 国際学会論文での発表（MLとLLMのハイブリッドアプローチの実証）
作成日: 2025-01-29

重要な改善点:
- 低確率領域（ML < 0.2）にFNの52%が存在する問題に対応
- 動的ブランドリスト（62個）を活用した検出
- LangGraphとStructured Outputのベストプラクティスを適用
"""

"""
セル番号: 01
概要: 環境設定とMLモデル結果の読み込み（偽陰性データの再構築を含む）
入力: 02_xgboost_training_evaluation.pyの出力（モデル、偽陰性データ、ブランドリスト）
出力: model, scaler, false_negatives_df, brand_keywords等のグローバル変数
"""

import json
import pickle
import joblib  # 02_xgboost_training_evaluation.pyはjoblibで保存
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import os
import glob
import psycopg2
from psycopg2.extras import RealDictCursor
import warnings
import xgboost as xgb
import subprocess
import re

# XGBoostのGPU警告を抑制
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost')

# ローカルGPUの確認（XGBoostモデル用）
GPU_AVAILABLE = True
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ ローカルGPU検出: XGBoostモデルでGPU利用可能")
    else:
        print("⚠️ ローカルGPU未検出: XGBoostはCPU処理に切り替えます")
        GPU_AVAILABLE = False
except:
    print("⚠️ nvidia-smi利用不可: XGBoostはCPU処理に切り替えます")
    GPU_AVAILABLE = False

# LLM設定（リモートvLLMサーバーを使用）
print("\n🤖 LLM設定:")
LLM_TYPE = "vllm"  # "ollama" or "vllm"
VLLM_BASE_URL = "http://192.168.100.71:30000/v1"  # リモートGPUサーバー
VLLM_MODEL = "Qwen/Qwen3-14B-FP8"  # 14Bパラメータモデル（サーバー側のGPUで実行）
OLLAMA_MODEL = 'qwen3:14b'  # フォールバック用

print(f"  - LLM実行環境: リモートvLLMサーバー（{VLLM_BASE_URL}）")
print(f"  - LLMモデル: {VLLM_MODEL}（14Bパラメータ）")
print(f"  - 注: LLMはリモートサーバーのGPUで実行されます")

# データベース設定
DB_CONFIG = {
    'dbname': 'rapids_data',
    'user': 'postgres',
    'password': 'asomura',
    'host': 'localhost',
    'port': '5432'
}

print("\n🔧 環境設定とデータ読み込み")
print("=" * 80)

# 開発モード確認
DEVELOPMENT_MODE = os.environ.get('DEV_MODE', 'false').lower() == 'true'
print(f"✅ 開発モード: {DEVELOPMENT_MODE}")

# ===== （差し替え）最新セッションIDの自動検出：02の仕様（%Y-%m-%d_%H%M%S）に忠実 =====
session_id = None

def _parse_session_id(name: str):
    m = re.match(r"^(\d{4})-(\d{2})-(\d{2})_(\d{6})$", name)
    if not m:
        return None
    try:
        return datetime.strptime(name, "%Y-%m-%d_%H%M%S")
    except ValueError:
        return None

models_dir = Path('models')
candidates = []

# まず 02 の正規形式フォルダを探索（xgboost_model.pkl と scaler.pkl が揃っているもののみ）
if models_dir.exists():
    for d in models_dir.iterdir():
        if d.is_dir():
            ts = _parse_session_id(d.name)
            if ts and (d / "xgboost_model.pkl").exists() and (d / "scaler.pkl").exists():
                candidates.append((ts, d))

if candidates:
    candidates.sort(key=lambda x: x[0])
    latest_session_dir = candidates[-1][1]
    session_id = latest_session_dir.name
    print(f"✅ models から 02形式の最新セッション検出: {session_id}")

# 互換フォールバック1：models/session_*（旧流儀）
if not session_id and models_dir.exists():
    session_dirs = [d for d in models_dir.iterdir() if d.is_dir() and d.name.startswith('session_')]
    if session_dirs:
        latest_session = max(session_dirs, key=lambda x: x.name)
        session_id = latest_session.name
        print(f"⚠️ 互換: models/session_* から検出: {session_id}")

# 互換フォールバック2：results/20*（結果フォルダ名から推定）
if not session_id:
    result_dirs = sorted(glob.glob('results/20*'))
    if result_dirs:
        latest_result_dir = result_dirs[-1]
        session_id = latest_result_dir.split('/')[-1]
        print(f"⚠️ 互換: results から検出: {session_id}")

if not session_id:
    raise ValueError("セッションが見つかりません。02_xgboost_training_evaluation.pyを先に実行してください。")

# ファイルパスの定義（以降は原文どおりの読み込みロジックに委ねる）
model_path = f"models/{session_id}/xgboost_model.pkl"
scaler_path = f"models/{session_id}/scaler.pkl"
feature_order_path = f"models/{session_id}/feature_order.json"
brand_keywords_path = f"models/{session_id}/brand_keywords.json"

print("  model_path :", model_path)
print("  scaler_path:", scaler_path)
print("  feature_order_path :", feature_order_path, ("(存在しません)" if not Path(feature_order_path).exists() else ""))
print("  brand_keywords_path:", brand_keywords_path, ("(存在しません)" if not Path(brand_keywords_path).exists() else ""))

# モデルの読み込み（joblib形式）
try:
    model = joblib.load(model_path)
    print(f"✅ XGBoostモデル読み込み完了: {model_path}")
except FileNotFoundError:
    print(f"❌ モデルファイルが見つかりません: {model_path}")
    print("   02_xgboost_training_evaluation.pyを実行してモデルを作成してください。")
    raise

# XGBoostモデルのGPU設定（ローカル実行、利用可能な場合）
if GPU_AVAILABLE:
    try:
        if hasattr(model, 'set_param'):
            model.set_param({'device': 'cuda:0'})
        elif hasattr(model, 'get_booster'):
            model.get_booster().set_param({'device': 'cuda:0'})
            if hasattr(model, 'set_params'):
                model.set_params(device='cuda:0', tree_method='hist')
        print(f"✅ XGBoostモデル: ローカルGPUを使用")
    except Exception as e:
        print(f"⚠️ GPU設定エラー: {e}")
        print("   XGBoostはCPU処理を継続します")

# スケーラーの読み込み（joblib形式）
try:
    scaler = joblib.load(scaler_path)
    print(f"✅ スケーラー読み込み完了: {scaler_path}")
except FileNotFoundError:
    print(f"❌ スケーラーファイルが見つかりません: {scaler_path}")
    print("   02_xgboost_training_evaluation.pyを実行してください。")
    raise

# 特徴量順序の読み込み（JSON形式）
try:
    with open(feature_order_path, 'r') as f:
        feature_order = json.load(f)
    print(f"✅ 特徴量順序読み込み完了: {len(feature_order)}個の特徴量")
except FileNotFoundError:
    print(f"❌ 特徴量順序ファイルが見つかりません: {feature_order_path}")
    print("   02_xgboost_training_evaluation.pyを実行してください。")
    raise

# 🔥 重要: 動的ブランドキーワードの読み込み（JSON形式）
try:
    with open(brand_keywords_path, 'r', encoding='utf-8') as f:
        brand_keywords = json.load(f)
    print(f"✅ ブランドキーワード読み込み完了: {len(brand_keywords)}個のブランド")
    print(f"   ブランド例: {brand_keywords[:5]}")
except FileNotFoundError:
    print(f"❌ ブランドキーワードファイルが見つかりません: {brand_keywords_path}")
    print("   02_xgboost_training_evaluation.pyを実行してブランドキーワードを生成してください。")
    print("   このファイルはGPT-4o miniで動的に生成される重要なデータです。")
    raise

# 偽陰性データの読み込み
false_negatives_df = None

# 新形式のファイルパスを試す
fn_paths = [
    f"models/{session_id}/false_negatives_reconstructed.pkl",
    f"results/{session_id}/false_negatives_reconstructed.pkl",
    f"results/{session_id}/false_negatives.pkl"
]

for fn_path in fn_paths:
    if os.path.exists(fn_path):
        try:
            # joblibで読み込み（02_xgboost_training_evaluation.pyの形式）
            fn_data = joblib.load(fn_path)
            
            # データ形式の判定
            if isinstance(fn_data, pd.DataFrame):
                false_negatives_df = fn_data
            elif isinstance(fn_data, dict):
                if 'analysis_df' in fn_data:
                    false_negatives_df = fn_data['analysis_df']
                elif 'domains' in fn_data and 'predictions' in fn_data:
                    # 旧形式の場合は再構築
                    false_negatives_df = pd.DataFrame({
                        'domain': fn_data['domains'],
                        'prediction_proba': fn_data['predictions'],
                        'source': fn_data.get('sources', ['unknown'] * len(fn_data['domains']))
                    })
            
            if false_negatives_df is not None:
                print(f"✅ 偽陰性データ読み込み完了: {fn_path}")
                break
                
        except Exception as e:
            print(f"⚠️ {fn_path}の読み込みに失敗: {e}")
            continue

if false_negatives_df is None:
    print("❌ 偽陰性データが見つかりません。")
    print("   02_xgboost_training_evaluation.pyを実行してデータを生成してください。")
    raise FileNotFoundError("偽陰性データファイルが見つかりません")

# 偽陰性データの統計表示
print(f"\n📊 偽陰性（FN）データの統計:")
print(f"  - 総FN数: {len(false_negatives_df):,}件")

if 'prediction_proba' in false_negatives_df.columns:
    print(f"  - 平均予測確率: {false_negatives_df['prediction_proba'].mean():.4f}")
    print(f"  - 中央値: {false_negatives_df['prediction_proba'].median():.4f}")
    print(f"  - 最小予測確率: {false_negatives_df['prediction_proba'].min():.4f}")
    print(f"  - 最大予測確率: {false_negatives_df['prediction_proba'].max():.4f}")
    
    # 🔥 低確率領域のFN分布（問題の核心）
    fn_low = false_negatives_df[false_negatives_df['prediction_proba'] < 0.2]
    fn_mid = false_negatives_df[(false_negatives_df['prediction_proba'] >= 0.2) & 
                                 (false_negatives_df['prediction_proba'] < 0.4)]
    fn_high = false_negatives_df[false_negatives_df['prediction_proba'] >= 0.7]
    print(f"\n🎯 低確率領域のFN分布（問題の核心）:")
    print(f"  - ML確率 < 0.2: {len(fn_low):,}件 ({len(fn_low)/len(false_negatives_df)*100:.1f}%)")
    print(f"  - ML確率 0.2-0.4: {len(fn_mid):,}件 ({len(fn_mid)/len(false_negatives_df)*100:.1f}%)")
    print(f"  - ML確率 >= 0.7: {len(fn_high):,}件 ({len(fn_high)/len(false_negatives_df)*100:.1f}%)")

# ソース別の統計
if 'source' in false_negatives_df.columns:
    print(f"\nソース別内訳:")
    source_counts = false_negatives_df['source'].value_counts()
    for source, count in source_counts.items():
        print(f"  - {source}: {count:,}件 ({count/len(false_negatives_df)*100:.1f}%)")

# 出力ディレクトリの作成
output_dirs = {
    'results': 'results/ai_agent_qwen3/',
    'logs': 'logs/ai_agent_qwen3/',
    'analysis': 'analysis/ai_agent_qwen3/'
}

for dir_name, dir_path in output_dirs.items():
    os.makedirs(dir_path, exist_ok=True)
    print(f"✅ {dir_name}ディレクトリ作成: {dir_path}")

# タイムスタンプ
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# グローバル変数（後のセルから参照）
cert_full_info_map = {}  # セル02で作成される
fn_features_df = None    # セル02で作成される

print(f"\n✅ 初期設定完了")
print(f"  - セッションID: {session_id}")
print(f"  - LLM実行: リモートvLLMサーバー（GPU使用）")
print(f"  - LLMモデル: {VLLM_MODEL if LLM_TYPE == 'vllm' else OLLAMA_MODEL}")
print(f"  - XGBoost: ローカル（{'GPU' if GPU_AVAILABLE else 'CPU'}使用）")
print(f"  - タイムスタンプ: {timestamp}")
print(f"  - 偽陰性件数: {len(false_negatives_df)}")
print(f"  - ブランド数: {len(brand_keywords)}")
print("=" * 80)

# 変数名の統一（セル07, 08との互換性のため）
ai_session_id = session_id
globals()['ai_session_id'] = ai_session_id
globals()['LLM_TYPE'] = LLM_TYPE
globals()['output_dirs'] = output_dirs
globals()['DEVELOPMENT_MODE'] = DEVELOPMENT_MODE
globals()['brand_keywords'] = brand_keywords  # 🔥 重要: グローバル変数として設定

print(f"\n🔧 変数名の統一完了:")
print(f"  - ai_session_id: {ai_session_id}")
print(f"  - brand_keywords: {len(brand_keywords)}個のブランドを利用可能")
print(f"  - LLM_TYPE: {LLM_TYPE}")
print(f"  - DEVELOPMENT_MODE: {DEVELOPMENT_MODE}")


# In[4]:


# === handoff (auto-added) ===
# このセルは自動追加: 次パートが必要とする最小集合のみを保存します（原文セルは無改変）。
import os, joblib
os.makedirs(HANDOFF_DIR, exist_ok=True)
handoff = {}
# --- keys: 'false_negatives_df', 'DB_CONFIG', 'brand_keywords'
handoff['false_negatives_df'] = false_negatives_df
handoff['DB_CONFIG'] = DB_CONFIG
handoff['brand_keywords'] = brand_keywords
joblib.dump(handoff, os.path.join(HANDOFF_DIR, "03_ai_agent_analysis_part1.pkl"))
print(f"✅ Saved handoff → {os.path.join(HANDOFF_DIR, '03_ai_agent_analysis_part1.pkl')}")


# In[ ]:


✅ IO guard ready -> artifacts/20251030_004055/handoff
✅ 02仕様に基づき最新セッションを自動選択:
  session_id  : 2025-10-22_082121
  model_path  : models/2025-10-22_082121/xgboost_model.pkl
  scaler_path : models/2025-10-22_082121/scaler.pkl
  feature_order_path : models/2025-10-22_082121/feature_order.json 
  brand_keywords_path: models/2025-10-22_082121/brand_keywords.json 
  results FN pkl: results/2025-10-22_082121/false_negatives.pkl


✅ ローカルGPU検出: XGBoostモデルでGPU利用可能

🤖 LLM設定:
  - LLM実行環境: リモートvLLMサーバー（http://192.168.100.71:30000/v1）
  - LLMモデル: Qwen/Qwen3-14B-FP8（14Bパラメータ）
  - 注: LLMはリモートサーバーのGPUで実行されます

🔧 環境設定とデータ読み込み
✅ 開発モード: False
✅ models から 02形式の最新セッション検出: 2025-10-22_082121
  model_path : models/2025-10-22_082121/xgboost_model.pkl
  scaler_path: models/2025-10-22_082121/scaler.pkl
  feature_order_path : models/2025-10-22_082121/feature_order.json 
  brand_keywords_path: models/2025-10-22_082121/brand_keywords.json 
✅ XGBoostモデル読み込み完了: models/2025-10-22_082121/xgboost_model.pkl
✅ XGBoostモデル: ローカルGPUを使用
✅ スケーラー読み込み完了: models/2025-10-22_082121/scaler.pkl
✅ 特徴量順序読み込み完了: 20個の特徴量
✅ ブランドキーワード読み込み完了: 100個のブランド
   ブランド例: ['allegro', 'facebook', 'microsoft', 'at&t', 'adobe']
✅ 偽陰性データ読み込み完了: results/2025-10-22_082121/false_negatives.pkl

📊 偽陰性（FN）データの統計:
  - 総FN数: 4,132件
  - 平均予測確率: 0.2086
  - 中央値: 0.1762
  - 最小予測確率: 0.0005
  - 最大予測確率: 0.4998

🎯 低確率領域のFN分布（問題の核心）:
  - ML確率 < 0.2: 2,241件 (54.2%)
  - ML確率 0.2-0.4: 1,315件 (3

In [4]:

# === セル3(パッチ): DB/LLM設定をConfigから取得 & GPU制御 (追記) ===
from typing import Dict, Any, Optional
import subprocess

def _summarize_db(db: Dict[str, Any]) -> Dict[str, Any]:
    return {k: db.get(k) for k in ("dbname","user","host","port")}

def apply_config_to_runtime(cfg: Dict[str, Any]) -> Dict[str, Any]:
    # --- DB ---
    db = cfg.get("db", {})
    timeout_s = int(db.get("timeout_s", 30))
    DB_CONFIG = {
        "dbname": str(db.get("dbname")),
        "user": str(db.get("user")),
        "password": str(db.get("password","")),
        "host": str(db.get("host")),
        "port": str(db.get("port")),
    }
    globals()["DB_CONFIG"] = DB_CONFIG
    globals()["DB_CONNECT_TIMEOUT_S"] = timeout_s
    print("🔧 DB設定:", _summarize_db(DB_CONFIG), f"(timeout_s={timeout_s})")

    # --- GPU detect ---
    GPU_AVAILABLE = False
    if cfg.get("system", {}).get("gpu_auto_detect", True):
        try:
            r = subprocess.run(["nvidia-smi"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            GPU_AVAILABLE = (r.returncode == 0)
            print("✅ GPU検出: 利用可能" if GPU_AVAILABLE else "⚠️ GPU未検出: CPUで実行します")
        except Exception:
            print("⚠️ nvidia-smi 実行不可: CPUで実行します")
    else:
        print("ℹ️ GPU自動検出は無効化 (cfg.system.gpu_auto_detect=False)")
    globals()["GPU_AVAILABLE"] = GPU_AVAILABLE

    # --- LLM ---
    try:
        from openai import OpenAI  # OpenAI SDK 互換 (vLLM/Ollama)
    except Exception:
        OpenAI = None

    llm_cfg = cfg.get("llm", {})
    provider = str(llm_cfg.get("provider","vllm")).lower()
    DEFAULT_MODEL = None
    client = None

    def _try_client(base_url: str, model: str, key: str = "EMPTY"):
        if OpenAI is None:
            return None
        try:
            cli = OpenAI(base_url=base_url, api_key=key or "EMPTY")
            _ = cli.chat.completions.create(model=model, messages=[{"role":"user","content":"ping"}], max_tokens=1, temperature=0.0)
            return cli
        except Exception:
            return None

    if provider == "vllm":
        base = str(llm_cfg.get("vllm_base_url") or "http://192.168.100.71:30000/v1")
        model = str(llm_cfg.get("vllm_model") or "Qwen/Qwen3-14B-FP8")
        client = _try_client(base, model)
        if client is None and llm_cfg.get("fallback_enabled", True):
            print("⚠️ vLLM接続に失敗。Ollamaへフォールバックを試みます")
            provider = "ollama"
        else:
            DEFAULT_MODEL = model

    if provider == "ollama":
        base = str(llm_cfg.get("ollama_base_url") or "http://localhost:11434/v1")
        model = str(llm_cfg.get("ollama_model") or "qwen3:14b")
        if client is None:
            client = _try_client(base, model, key="ollama")
        DEFAULT_MODEL = model

    LLM_READY = (client is not None) if OpenAI is not None else False
    if OpenAI is None:
        print("ℹ️ openai パッケージ未導入: LLMは無効化（フォールバックで継続）")

    globals()["LLM_CLIENT"] = client
    globals()["LLM_READY"] = LLM_READY
    globals()["LLM_TYPE"]  = provider
    globals()["DEFAULT_MODEL"] = DEFAULT_MODEL
    globals()["VLLM_BASE_URL"] = llm_cfg.get("vllm_base_url")
    globals()["OLLAMA_BASE_URL"] = llm_cfg.get("ollama_base_url")
    print(f"🤖 LLM: type={provider}, ready={LLM_READY}, model={DEFAULT_MODEL}")

    return {
        "DB_CONFIG": DB_CONFIG,
        "GPU_AVAILABLE": GPU_AVAILABLE,
        "LLM_TYPE": provider,
        "LLM_READY": LLM_READY,
        "DEFAULT_MODEL": DEFAULT_MODEL,
    }

# （元セル3の後でも確実に適用するための再適用フック）
if "cfg" in globals() and isinstance(cfg, dict):
    _ = apply_config_to_runtime(cfg)


In [5]:

# === セルX: 偽陰性分析の閾値をConfigから取得 (追記) ===
from typing import Dict, Any
import pandas as pd, numpy as np

def _get_thresholds(cfg: Dict[str, Any]) -> Dict[str, float]:
    a = cfg.get("analysis", {})
    low = float(a.get("fn_threshold_low", 0.2))
    high = float(a.get("fn_threshold_high", 0.7))
    detailed = bool(a.get("enable_detailed_stats", True))
    return {"low": low, "high": high, "detailed": detailed}

def analyze_low_prob_region(false_negatives_df: pd.DataFrame, cfg: Dict[str, Any]) -> Dict[str, Any]:
    th = _get_thresholds(cfg)
    low = th["low"]; high = th["high"]; detailed = th["detailed"]
    if false_negatives_df is None or "prediction_proba" not in false_negatives_df.columns:
        print("⚠️ false_negatives_df 未準備、または prediction_proba 列がありません")
        return {"count": 0, "low_threshold": low, "high_threshold": high}

    df = false_negatives_df
    low_df = df[df["prediction_proba"] < low]
    mid_df = df[(df["prediction_proba"] >= low) & (df["prediction_proba"] < high)]
    high_df = df[df["prediction_proba"] >= high]

    print(f"🎯 低確率領域分析（しきい値 low={low}, high={high}）")
    print(f"  - ML確率 < {low:.2f}: {len(low_df):,}件")
    if detailed:
        print(f"  - {low:.2f} <= ML確率 < {high:.2f}: {len(mid_df):,}件")
        print(f"  - ML確率 >= {high:.2f}: {len(high_df):,}件")

    stats = {
        "low_region_count": int(len(low_df)),
        "mid_region_count": int(len(mid_df)),
        "high_region_count": int(len(high_df)),
        "low_threshold": float(low),
        "high_threshold": float(high),
    }
    if detailed and len(low_df) > 0:
        stats.update({
            "low_mean": float(low_df["prediction_proba"].mean()),
            "low_median": float(low_df["prediction_proba"].median()),
        })
    return stats

# 閾値をグローバルに公開
if "cfg" in globals():
    __th = _get_thresholds(cfg)
    FN_THRESHOLD_LOW = __th["low"]; FN_THRESHOLD_HIGH = __th["high"]; ENABLE_DETAILED_STATS = __th["detailed"]
    globals().update({"FN_THRESHOLD_LOW": FN_THRESHOLD_LOW, "FN_THRESHOLD_HIGH": FN_THRESHOLD_HIGH, "ENABLE_DETAILED_STATS": ENABLE_DETAILED_STATS})
    print(f"🔎 閾値を設定: low={FN_THRESHOLD_LOW}, high={FN_THRESHOLD_HIGH}, detailed={ENABLE_DETAILED_STATS}")


In [6]:

# === 最終セル: Controller API関数 (agent_minimal) (追記) ===
from typing import Tuple, Dict, Any, Optional, List
import os, json, joblib, traceback
from pathlib import Path

def _find_first(*candidates: str) -> Optional[str]:
    for p in candidates:
        if p and os.path.exists(p):
            return p
    return None

def _load_false_negatives(session_id: str):
    import pandas as pd
    cands = [
        f"results/{session_id}/false_negatives_reconstructed.pkl",
        f"models/{session_id}/false_negatives_reconstructed.pkl",
        f"results/{session_id}/false_negatives.pkl",
    ]
    p = _find_first(*cands)
    if not p:
        return None
    try:
        obj = joblib.load(p)
        if isinstance(obj, pd.DataFrame):
            return obj
        if isinstance(obj, dict):
            if "analysis_df" in obj and isinstance(obj["analysis_df"], pd.DataFrame):
                return obj["analysis_df"]
            if "domains" in obj and "predictions" in obj:
                return pd.DataFrame({"domain": obj["domains"], "prediction_proba": obj["predictions"], "source": obj.get("sources")})
    except Exception:
        return None
    return None

def _load_brand_keywords(session_id: str) -> List[str]:
    p1 = f"models/{session_id}/brand_keywords.json"
    if os.path.exists(p1):
        try:
            return json.loads(Path(p1).read_text(encoding='utf-8'))
        except Exception:
            pass
    try:
        models_dir = Path("models")
        dirs = sorted([d for d in models_dir.glob("*") if d.is_dir()])
        for d in reversed(dirs):
            p = d / "brand_keywords.json"
            if p.exists():
                return json.loads(p.read_text(encoding='utf-8'))
    except Exception:
        pass
    return []

def _ensure_dirs(base: Path) -> Dict[str, str]:
    results_dir = base / "results"
    handoff_dir = base / "handoff"
    results_dir.mkdir(parents=True, exist_ok=True)
    handoff_dir.mkdir(parents=True, exist_ok=True)
    return {"results_dir": str(results_dir), "handoff_dir": str(handoff_dir)}

def _simple_risk_correction(df, brand_keywords: List[str], low_th: float):
    import pandas as pd
    df = df.copy()
    if "prediction_proba" not in df.columns or "domain" not in df.columns:
        return pd.DataFrame(columns=["domain","prediction_proba","corrected","reason"])
    brands = [b.lower() for b in (brand_keywords or [])]
    def _has_brand(d: str) -> bool:
        d = str(d).lower()
        return any(b and b in d for b in brands[:200])
    def _tld(d: str) -> str:
        parts = str(d).lower().split(".")
        return parts[-1] if len(parts) > 1 else ""
    def _short(d: str) -> bool:
        left = str(d).split(".")[0]
        return len(left) < 10
    corrected, reasons = [], []
    for _, row in df.iterrows():
        ml = float(row.get("prediction_proba", 1.0)); d = str(row.get("domain",""))
        c = False; reason = []
        if ml < low_th:
            if _has_brand(d): c=True; reason.append("brand+lowML")
            if _short(d):     c=True; reason.append("short+lowML")
            tl = _tld(d)
            if tl in ("tk","gq","ml","cf","ga"):
                c=True; reason.append(f"tld({tl})+lowML")
        corrected.append(c); reasons.append(",".join(reason))
    df["corrected"] = corrected; df["reason"] = reasons
    return df

def agent_minimal(session_id: str, inputs: Dict[str, Any] = {"mode":"fn"}, cfg: Dict[str, Any] = None) -> Tuple[str, Dict[str, str]]:
    import pandas as pd
    import matplotlib.pyplot as plt
    from datetime import datetime
    try:
        if not isinstance(session_id, str) or not session_id.strip():
            return "INVALID_INPUT", {"error": "session_idは必須です"}

        local_cfg = load_configuration(cfg_override=(cfg or {}))
        _ = apply_config_to_runtime(local_cfg)
        if local_cfg.get("system", {}).get("cert_only_mode", False):
            print("⚠️ 注意: cert_only_modeですが、ブランド偽装検出は研究の核心機能のため実行します")

        base = Path("artifacts") / session_id
        dirs = _ensure_dirs(base)
        results_dir = Path(dirs["results_dir"]); handoff_dir = Path(dirs["handoff_dir"])

        brand_keywords = _load_brand_keywords(session_id)
        if len(brand_keywords) < 1:
            print("⚠️ ブランドキーワードが見つかりません。基本分析のみ実行します")
        elif len(brand_keywords) < 62:
            print(f"⚠️ ブランドキーワードが{len(brand_keywords)}件 (>=62が推奨)")

        fn_df = _load_false_negatives(session_id)
        if fn_df is None or fn_df.empty:
            return "NOT_FOUND", {"error": "偽陰性データが見つかりません", "session_id": session_id}

        low_th = float(local_cfg.get("analysis", {}).get("fn_threshold_low", 0.2))
        corr_df = _simple_risk_correction(fn_df, brand_keywords, low_th)

        low_df = fn_df[fn_df["prediction_proba"] < low_th]
        corrected_low = corr_df[(corr_df["prediction_proba"] < low_th) & (corr_df["corrected"] == True)]
        improvement_rate = (len(corrected_low) / max(1, len(low_df))) * 100.0

        plt.figure(figsize=(6,4))
        bars = [len(low_df)-len(corrected_low), len(corrected_low)]
        plt.bar(["未補正","補正"], bars)
        plt.title(f"低確率領域(<{low_th:.2f})の補正数 (改善率 {improvement_rate:.1f}%)")
        plt.ylabel("件数")
        out_improve = Path(results_dir) / "fn_detection_improvement.png"
        plt.tight_layout(); plt.savefig(out_improve, dpi=150); plt.close()

        plt.figure(figsize=(6,4))
        fn_df["prediction_proba"].hist(bins=30)
        try:
            plt.axvline(x=low_th, color="red", linestyle="--", label=f"low={low_th}")
            plt.legend()
        except Exception:
            pass
        plt.title("偽陰性の予測確率分布")
        out_dist = Path(results_dir) / "ml_probability_distribution.png"
        plt.tight_layout(); plt.savefig(out_dist, dpi=150); plt.close()

        if "corrected" in corr_df.columns:
            top_reasons = corr_df[corr_df["corrected"]==True]["reason"].value_counts()[:5]
            plt.figure(figsize=(6,4))
            top_reasons.plot(kind="bar")
            plt.title("補正の主因 (上位)")
            out_brand = Path(results_dir) / "brand_detection_results.png"
            plt.tight_layout(); plt.savefig(out_brand, dpi=150); plt.close()
        else:
            out_brand = Path(results_dir) / "brand_detection_results.png"; out_brand.touch()

        report = {
            "session_id": session_id,
            "created_at": datetime.now().isoformat(),
            "metrics": {
                "low_threshold": low_th,
                "low_region_count": int(len(low_df)),
                "corrected_low_count": int(len(corrected_low)),
                "improvement_rate_pct": float(improvement_rate),
            },
            "notes": [
                "LangGraph未使用の最小フォールバック",
                "vLLM→Ollama フォールバックは apply_config_to_runtime で実施",
            ],
        }
        out_json = Path(results_dir) / "agent_performance_report.json"
        out_json.write_text(json.dumps(report, indent=2, ensure_ascii=False), encoding="utf-8")

        out_corr = Path(results_dir) / "false_negatives_corrected.csv"
        corr_df.to_csv(out_corr, index=False, encoding="utf-8")

        handoff_obj = {"false_negatives_df": fn_df, "corrected_df": corr_df, "brand_keywords": brand_keywords, "cfg": local_cfg}
        out_handoff = Path(dirs["handoff_dir"]) / "03_ai_agent_analysis_part1.pkl"
        joblib.dump(handoff_obj, out_handoff)

        Paths = {
            "fn_detection_improvement": str(out_improve),
            "ml_probability_distribution": str(out_dist),
            "brand_detection_results": str(out_brand),
            "agent_performance_report": str(out_json),
            "false_negatives_corrected": str(out_corr),
            "handoff": str(out_handoff),
        }
        return "OK", Paths
    except Exception as e:
        tb = traceback.format_exc(limit=2)
        return "ERROR", {"error": str(e), "traceback": tb}
